In [13]:
import ee
import geemap

In [14]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [15]:
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [16]:
s2 = (ee.ImageCollection('COPERNICUS/S2')
     .map(lambda image : image.clip(brma2_region)))

In [22]:
#Create cloud mask; hasn't been implemented yet
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()

def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask).divide(10000)

cloudMasked = s2.filterBounds(brma2_region).map(maskS2clouds);

median = cloudMasked.median();
Map.addLayer(median, {'bands': ['B4', 'B3', 'B2'], 'max': .3}, 'median')